In [21]:
import torch
import torch.nn.functional as F

In [22]:
a = torch.randn(32, 350, 100)
b = torch.randn(32, 350, 100)

In [26]:
F.cosine_similarity(a,b,dim=2)

tensor([[ 0.1209,  0.0109, -0.1487,  ..., -0.0915,  0.0144,  0.0532],
        [-0.0186,  0.1274,  0.0586,  ...,  0.0165,  0.0213,  0.0094],
        [ 0.0227,  0.1961, -0.0272,  ..., -0.0813,  0.0528,  0.1152],
        ...,
        [-0.0361, -0.0095,  0.1007,  ...,  0.0678, -0.0797, -0.0607],
        [-0.1285, -0.1127, -0.0833,  ...,  0.0406,  0.0003, -0.1108],
        [ 0.0199, -0.1339,  0.0618,  ...,  0.0854,  0.2396,  0.1192]])